In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

### 1. Introduction

The aim of this analysis is to determine which albums from a new record label deal should be added to the Chinook store. The four options are all different genres of music and come from different artists.

A schema diagram is shown below for reference:

In [31]:
%%sql

SELECT type,name
FROM sqlite_master
WHERE type IN ("table","view")

 * sqlite:///chinook.db
Done.


type,name
table,album
table,artist
table,customer
table,employee
table,genre
table,invoice
table,invoice_line
table,media_type
table,playlist
table,playlist_track


### 2. US sales by genre

Since all the album choices are of different genres, it will be the main differentiating factor to consider. Thus, we should determine which genres are the most popular and sell the most. Since Chinook is primarily concerned with the US market, the data will be filtered for US sales only.

In [8]:
%%sql

SELECT g.name AS genre,
COUNT(*) AS sales,
ROUND(CAST(COUNT(*) AS float)/
(SELECT COUNT(*) FROM invoice AS i
LEFT JOIN invoice_line AS il ON i.invoice_id=il.invoice_id
WHERE billing_country="USA"),3) AS percentage_sales
FROM invoice AS i

LEFT JOIN invoice_line AS il ON i.invoice_id=il.invoice_id
LEFT JOIN track AS t ON il.track_id=t.track_id
LEFT JOIN genre AS g ON t.genre_id=g.genre_id

WHERE i.billing_country="USA"
GROUP BY g.name
ORDER BY sales DESC

 * sqlite:///chinook.db
Done.


genre,sales,percentage_sales
Rock,561,0.534
Alternative & Punk,130,0.124
Metal,124,0.118
R&B/Soul,53,0.05
Blues,36,0.034
Alternative,35,0.033
Pop,22,0.021
Latin,22,0.021
Hip Hop/Rap,20,0.019
Jazz,14,0.013


We can see that the "Rock" genre is the most popular and accounts for over half of total sales in the US, with a sharp dropoff for the next most popular genres ("Alternative & Punk", "Metal", etc.)

### 3. Sales performance by agent

Another potential factor to consider is sales performance by support agent.

In [21]:
%%sql

SELECT e.last_name || ", " || e.first_name AS name,
e.hire_date AS hire_date,
CAST(SUM(i.total) AS int) AS sales
FROM invoice AS i

LEFT JOIN customer AS c ON i.customer_id=c.customer_id
LEFT JOIN employee AS e ON c.support_rep_id=e.employee_id

GROUP BY c.support_rep_id

 * sqlite:///chinook.db
Done.


name,hire_date,sales
"Peacock, Jane",2017-04-01 00:00:00,1731
"Park, Margaret",2017-05-03 00:00:00,1584
"Johnson, Steve",2017-10-17 00:00:00,1393


Here we can see that Jane has made the most sales and Steve the least, although this corresponds to their respective tenures at Chinook, so performance is roughly equal among each agent and thus not a differentiating factor. To confirm this, we could consider sales per month of tenure rather than raw sales.

### 4. Sales by country

Another important factor to consider if there are any opportunities in other markets, so we pull the total number of customers, total sales, average sales per customer, and average order size for each country.

In [29]:
%%sql

SELECT
country_2 AS country,
COUNT(DISTINCT customer_id) AS total_customers,
CAST(SUM(total) AS int) AS total_sales,
CAST(SUM(total)/COUNT(DISTINCT customer_id) AS int) AS avg_sales,
ROUND(SUM(total)/COUNT(invoice_id),1) AS avg_order
FROM

(SELECT
CASE 
    WHEN (SELECT COUNT(*) FROM customer WHERE country=c.country)=1 THEN "Other"
    ELSE c.country
    END AS country_2,
c.customer_id,
i.total,
i.invoice_id,
CASE
    WHEN (SELECT COUNT(*) FROM customer WHERE country=c.country)=1 THEN 1
    ELSE 0
    END AS other_sort

FROM customer AS c

LEFT JOIN invoice AS i ON c.customer_id=i.customer_id)

GROUP BY 1
ORDER BY other_sort ASC, total_customers DESC

 * sqlite:///chinook.db
Done.


country,total_customers,total_sales,avg_sales,avg_order
USA,13,1040,80,7.9
Canada,8,535,66,7.0
Brazil,5,427,85,7.0
France,5,389,77,7.8
Germany,4,334,83,8.2
United Kingdom,3,245,81,8.8
Czech Republic,2,273,136,9.1
India,2,183,91,8.7
Portugal,2,185,92,6.4
Other,15,1094,72,7.4


Based on our analysis above, there could be opportunity in the Czech Republic due to the higher average sales per customer. 

Some ways to improve analysis:
* Increase sample size
* Benchmark total customers to population size - smaller markets that have high average sales per customer or average order size can appear to be better opportunities than they are if they the market cannot support additional sales.

### 5. Albums versus individual tracks

Currently the Chinook store allows customers to make purchases in one of two ways:
* Purchase a whole album
* Purchase collection of one or more individual tracks

Management is considering purchasing only the most popular tracks from an album rather than purchasing every track from an album. Thus, we need to determine the proportion of purchases that are individual tracks versus whole albums.

In [30]:
%%sql

WITH invoice_final AS 
(SELECT il.invoice_id AS invoice_id,
MIN(il.track_id) AS min_track_id
FROM invoice_line AS il
GROUP BY 1)

SELECT album_purchased,
COUNT(invoice_id) AS no_invoices,
ROUND(CAST(COUNT(invoice_id) AS float)/
(SELECT COUNT(*) FROM invoice),3) AS percentage_invoices

FROM
(SELECT if.*,
CASE
    WHEN 
        (SELECT track_id FROM invoice_line
        WHERE invoice_id=if.invoice_id
        EXCEPT
        SELECT track_id FROM track
        WHERE album_id=
        (SELECT album_id FROM track WHERE track_id=if.min_track_id)) 
        IS NULL
    AND
        (SELECT track_id FROM track
        WHERE album_id=
        (SELECT album_id FROM track WHERE track_id=if.min_track_id)
        EXCEPT
        SELECT track_id FROM invoice_line
        WHERE invoice_id=if.invoice_id)
        IS NULL
    THEN "yes"
    ELSE "no"
    END AS album_purchased

FROM invoice_final AS if)

GROUP BY 1;

 * sqlite:///chinook.db
Done.


album_purchased,no_invoices,percentage_invoices
no,500,0.814
yes,114,0.186


From our analysis, we can see that a decent portion (18.6%) of purchases are of whole albums, so only purchasing the most popular albums could result in a loss of up to that proportion of revenue.